# Accessing Higher Energy States

ほとんどの量子アルゴリズム/アプリケーションでは、|0>と|1>の2次元空間で計算が実行されます。ただし、IBMのハードウェアでは、通常は使用されない、より高いエネルギー状態も存在します。このセクションの焦点は、Qiskit Pulseを使用してこれらの状態を探索することです。特に、|2>状態を励起し、|0>、|1>及び|2>状態を分類するための弁別子を作成する方法を示します。

このノートブックを読む前に、前の章を確認することをお勧めします。また、Qiskit Pulse specificationsを読むことをお勧めします。

#### Physics Background

ここで、IBMの量子ハードウェアの多くの基礎である、トランスモンキュービットの物理学に関する追加の背景を説明します。これらのシステムには、ジェセフソン接合とコンデンサーで構成される超電導回路が含まれています。超電導回路に不慣れな方は、こちらレビューを参照してください。<br>
https://arxiv.org/pdf/1904.06560.pdf<br>
このシステムのハミルトニアンは、

$H=4E_cN^2 - E_Jcos(φ)$

ここで、$E_C$と$E_J$はコンデンサとジョセフソンエネルギーを表し、$n$は電荷数の演算子を減らし、$φ$は接合部全体の磁束を減らします。$ℏ=1$の単位で作業します。

Transmon qubitは$φ$が小さい領域で定義されるため、テイラー級数で$E_Jcos(φ)$を展開できます(定数項を無視)

$E_Jcos(φ) ≈ \frac{1}{2}E_Jφ^2 - \frac{1}{24}E_Jφ^4 + Ο(φ^6)$

二次項$φ^2$は標準調和振動子を定義します。追加の各用語は非調和性をもたらします。

$n ~ (a - a†)$, $φ ~ (a + a†)$(上げ演算子、下げ演算子$a†$,$a$)を使用して、システムがハミルトニアンのダッフィング発振器に似ていることを示すことができます。

$H = ωa†a+\frac{a}{2}a†a†aa$,

ここでは、$ω$は0→1励起周波数($ω≡ω^{0→1}$)を与え、$α$は0→1と1→2周波数の間の非調和性($α≡ω^{1→2} - ω^{0→1}$)です。必要に応じてドライブ条件を追加できます。

標準の2次元部分空間に特化することを選択した場合、$|α|$を作成できます。十分に大きいか、特別な制御技術を使用して、より高いエネルギー状態を抑制します。

## 0. Getting Started

まず、依存関係をインポートし、いくつかのデフォルトの変数値を定義します。実験を実行するには、$qubit_0$を選択します。実験は、公に利用可能な単一のqubitデバイス"ibmq_armonk"で行います。

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit
from scipy.signal import find_peaks

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split

import qiskit.pulse as pulse
import qiskit.pulse.pulse_lib as pulse_lib
from qiskit.compiler import assemble
from qiskit.pulse.commands import SamplePulse

from qiskit.tools.monitor import job_monitor

In [2]:
import warnings
warnings.filterwarnings('ignore')
from qiskit.tools.jupyter import *
%matplotlib inline

from qiskit import IBMQ
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_armonk')

backend_config = backend.configuration()
assert backend_config.open_pulse, "Backend doesn't support Pulse"

dt = backend_config.dt

backend_defaults = backend.defaults()

# unit conversion factors -> all backend properties returned in SI (Hz, sec, etc)
GHz = 1.0e9 # Gigahertz
MHz = 1.0e6 # Megahertz
us = 1.0e-6 # Microseconds
ns = 1.0e-9 # Nanoseconds

qubit = 0 # qubit we will analyze
default_qubit_freq = backend_defaults.qubit_freq_est[qubit] # Default qubit frequency in Hz. 
print(f"Qubit {qubit} has an estimated frequency of {default_qubit_freq/ GHz} GHz.")

# scale data (specific to each device)
scale_factor = 1e-14

# number of shots for our experiments
NUM_SHOTS = 1024

### Collect the necessary channels
drive_chan = pulse.DriveChannel(qubit)
meas_chan = pulse.MeasureChannel(qubit)
acq_chan = pulse.AcquireChannel(qubit)

Qubit 0 has an estimated frequency of 4.974441106581355 GHz.


追加のヘルパー関数をいくつか定義します。

In [3]:
def get_job_data(job, average):
    """Retrieve data from a job that has already run.
    Args:
        job (Job): The job whose data you want.
        average (bool): If True, gets the data assuming data is an average.
                        If False, gets the data assuming it is for single shots.
    Return:
        list: List containing job result data. 
    """
    job_results = job.result(timeout=120) # timeout parameter set to 120 s
    result_data = []
    for i in range(len(job_results.results)):
        if average: # get avg data
            result_data.append(job_results.get_memory(i)[qubit]*scale_factor) 
        else: # get single data
            result_data.append(job_results.get_memory(i)[:, qubit]*scale_factor)  
    return result_data

def get_closest_multiple_of_16(num):
    """Compute the nearest multiple of 16. Needed because pulse enabled devices require 
    durations which are multiples of 16 samples.
    """
    return (int(num) - (int(num)%16))

次に、駆動パルスと測定のいくつかのデフォルトパラメータを含めます。命令スケジュールマップ(バックエンドのデフォルトから)からメジャーコマンドをプルして、新しいキャリブレーションで更新されるようにします。

In [4]:
# Drive pulse parameters (us = microseconds)
drive_sigma_us = 0.075                     # This determines the actual width of the gaussian
drive_samples_us = drive_sigma_us*8        # This is a truncating parameter, because gaussians don't have 
                                           # a natural finite length

drive_sigma = get_closest_multiple_of_16(drive_sigma_us * us /dt)       # The width of the gaussian in units of dt
drive_samples = get_closest_multiple_of_16(drive_samples_us * us /dt)   # The truncating parameter in units of dt

In [5]:
# Find out which measurement map index is needed for this qubit
meas_map_idx = None
for i, measure_group in enumerate(backend_config.meas_map):
    if qubit in measure_group:
        meas_map_idx = i
        break
assert meas_map_idx is not None, f"Couldn't find qubit {qubit} in the meas_map!"

In [6]:
# Get default measurement pulse from instruction schedule map
inst_sched_map = backend_defaults.instruction_schedule_map
measure = inst_sched_map.get('measure', qubits=backend_config.meas_map[meas_map_idx])

## 1. Discriminating the |0> and |1> states

このセクションでは、標準の|0>及び|1>の状態の弁別子(discriminator)を作成します。弁別器の役割は、meas_level = 1の複雑なデータを取得し、それを標準の|0>及び|1>の状態(meas_level = 2)に分類することです。これは、前の章の作業の多くを複製します。これらの結果は、このノートブックの焦点である高エネルギー状態を励起するために必要です。

### 1A. 0->1 Frequency Sweep

弁別器を構築する最初のステップは、前の章で行ったように、qubit周波数をキャリブレートすることです。

In [7]:
def create_ground_freq_sweep_program(freqs, drive_power):
    """Builds a program that does a freq sweep by exciting the ground state. 
    Depending on drive power this can reveal the 0->1 frequency or the 0->2 frequency. 
    Args:
        freqs (np.ndarray(dtype=float)): Numpy array of frequencies to sweep.
        drive_power (float) : Value of drive amplitude.
    Raises:
        ValueError: Raised if use more than 75 frequencies; currently, an error will be thrown on the backend 
                    if you try to do this.
    Returns:
        Qobj: Program for ground freq sweep experiment.
    """
    if len(freqs) > 75:
        raise ValueError("You can only run 75 schedules at a time.")
    
    # print information on the sweep
    print(f"The frequency sweep will go from {freqs[0] / GHz} GHz to {freqs[-1]/ GHz} GHz \
using {len(freqs)} frequencies. The drive power is {drive_power}.")
    
    # Define the drive pulse
    ground_sweep_drive_pulse = pulse_lib.gaussian(duration=drive_samples,
                                                  sigma=drive_sigma,
                                                  amp=drive_power,
                                                  name='ground_sweep_drive_pulse')
    # Create the base schedule
    schedule = pulse.Schedule(name='Frequency sweep starting from ground state.')
    
    schedule |= ground_sweep_drive_pulse(drive_chan)
    schedule |= measure << schedule.duration
    
    # define frequencies for the sweep
    schedule_freqs = [{drive_chan: freq} for freq in freqs]

    # assemble the program
    # Note: we only require a single schedule since each does the same thing;
    # for each schedule, the LO frequency that mixes down the drive changes
    # this enables our frequency sweep
    ground_freq_sweep_program = assemble(schedule,
                                         backend=backend, 
                                         meas_level=1,
                                         meas_return='avg',
                                         shots=NUM_SHOTS,
                                         schedule_los=schedule_freqs)
    
    return ground_freq_sweep_program

In [8]:
# We will sweep 40 MHz around the estimated frequency, with 75 frequencies
num_freqs = 75
ground_sweep_freqs = default_qubit_freq + np.linspace(-20*MHz, 20*MHz, num_freqs)
ground_freq_sweep_program = create_ground_freq_sweep_program(ground_sweep_freqs, drive_power=0.3)

The frequency sweep will go from 4.954441106581355 GHz to 4.994441106581355 GHz using 75 frequencies. The drive power is 0.3.


In [9]:
ground_freq_sweep_job = backend.run(ground_freq_sweep_program)

In [10]:
print(ground_freq_sweep_job.job_id())
job_monitor(ground_freq_sweep_job)

5f3ec48244190600125adbc7
Job Status: job has successfully run


In [11]:
# Get the job data (average)
ground_freq_sweep_data = get_job_data(ground_freq_sweep_job, average=True)

データをローレンツ曲線に適合させ、キャリブレーションされた周波数を抽出します。

In [12]:
def fit_function(x_values, y_values, function, init_params):
    """Fit a function using scipy curve_fit."""
    fitparams, conv = curve_fit(function, x_values, y_values, init_params)
    y_fit = function(x_values, *fitparams)
    
    return fitparams, y_fit

In [13]:
# do fit in Hz
(ground_sweep_fit_params, 
 ground_sweep_y_fit) = fit_function(ground_sweep_freqs,
                                   ground_freq_sweep_data, 
                                   lambda x, A, q_freq, B, C: (A / np.pi) * (B / ((x - q_freq)**2 + B**2)) + C,
                                   [7, 4.975*GHz, 1*GHz, 3*GHz] # initial parameters for curve_fit
                                   )

RuntimeError: Optimal parameters not found: Number of calls to function has reached maxfev = 1000.

In [ ]:
# Note: we are only plotting the real part of the signal
plt.scatter(ground_sweep_freqs/GHz, ground_freq_sweep_data, color='black')
plt.plot(ground_sweep_freqs/GHz, ground_sweep_y_fit, color='red')
plt.xlim([min(ground_sweep_freqs/GHz), max(ground_sweep_freqs/GHz)])
plt.xlabel("Frequency [GHz]", fontsize=15)
plt.ylabel("Measured Signal [a.u.]", fontsize=15)
plt.title("0->1 Frequency Sweep", fontsize=15)
plt.show()

In [ ]:
_, cal_qubit_freq, _, _ = ground_sweep_fit_params
print(f"We've updated our qubit frequency estimate from "
      f"{round(default_qubit_freq/GHz, 7)} GHz to {round(cal_qubit_freq/GHz, 7)} GHz.")

### 1B. 0->1 Rabi Experiment

次に、Rabi実験を実行して$0→1π$パルス振幅を計算します。覚えておいてください、$π$パルスは、|0>から|1>の状態(ブロッホ球上の$π回転$)へと進むパルスです。

In [ ]:
# experimental configuration
num_rabi_points = 50 # number of experiments (ie amplitudes to sweep out)

# Drive amplitude values to iterate over: 50 amplitudes evenly spaced from 0 to 0.75
drive_amp_min = 0
drive_amp_max = 0.75
drive_amps = np.linspace(drive_amp_min, drive_amp_max, num_rabi_points)

In [ ]:
# Create schedule
rabi_01_schedules = []
# loop over all drive amplitudes
for ii, drive_amp in enumerate(drive_amps):
    # drive pulse
    rabi_01_pulse = pulse_lib.gaussian(duration=drive_samples, 
                                       amp=drive_amp, 
                                       sigma=drive_sigma, 
                                       name='rabi_01_pulse_%d' % ii)
    
    # add commands to schedule
    schedule = pulse.Schedule(name='Rabi Experiment at drive amp = %s' % drive_amp)
    schedule |= rabi_01_pulse(drive_chan)
    schedule |= measure << schedule.duration # shift measurement to after drive pulse
    rabi_01_schedules.append(schedule)

In [ ]:
# Assemble the schedules into a program
# Note: We drive at the calibrated frequency.
rabi_01_expt_program = assemble(rabi_01_schedules,
                                backend=backend,
                                meas_level=1,
                                meas_return='avg',
                                shots=NUM_SHOTS,
                                schedule_los=[{drive_chan: cal_qubit_freq}]
                                               * num_rabi_points)

In [ ]:
rabi_01_job = backend.run(rabi_01_expt_program)

In [ ]:
print(rabi_01_job.job_id())
job_monitor(rabi_01_job)

In [ ]:
# Get the job data (average)
rabi_01_data = get_job_data(rabi_01_job, average=True)

In [ ]:
def baseline_remove(values):
    """Center data around 0."""
    return np.array(values) - np.mean(values)

In [ ]:
# Note: Only real part of data is plotted
rabi_01_data = np.real(baseline_remove(rabi_01_data))
(rabi_01_fit_params, 
 rabi_01_y_fit) = fit_function(drive_amps,
                               rabi_01_data, 
                               lambda x, A, B, drive_01_period, phi: (A*np.cos(2*np.pi*x/drive_01_period - phi) + B),
                               [4, -4, 0.5, 0])

plt.scatter(drive_amps, rabi_01_data, color='black')
plt.plot(drive_amps, rabi_01_y_fit, color='red')

drive_01_period = rabi_01_fit_params[2] 
# account for phi in computing pi amp
pi_amp_01 = (drive_01_period/2/np.pi) *(np.pi+rabi_01_fit_params[3])

plt.axvline(pi_amp_01, color='red', linestyle='--')
plt.axvline(pi_amp_01+drive_01_period/2, color='red', linestyle='--')
plt.annotate("", xy=(pi_amp_01+drive_01_period/2, 0), xytext=(pi_amp_01,0), arrowprops=dict(arrowstyle="<->", color='red'))
plt.annotate("$\pi$", xy=(pi_amp_01-0.03, 0.1), color='red')

plt.xlabel("Drive amp [a.u.]", fontsize=15)
plt.ylabel("Measured signal [a.u.]", fontsize=15)
plt.title('0->1 Rabi Experiment', fontsize=15)
plt.show()

In [ ]:
print(f"Pi Amplitude (0->1) = {pi_amp_01}")

これらの結果を使用して、$0→1π$パルスを定義します。

In [ ]:
pi_pulse_01 = pulse_lib.gaussian(duration=drive_samples,
                                 amp=pi_amp_01, 
                                 sigma=drive_sigma,
                                 name='pi_pulse_01')

### 1C. Build the 0,1 discriminator

これでキャリブレーションされた周波数と$π$パルスが得られたので、|0>と|1>の状態の弁別子を作成できます。弁別器は、IQプレーンのmeas_level = 1データを取得し、それを|0>または|1>に分類することによって機能します。

|0>と|1>の状態は、重心として知られているIQ平面でコヒーレントな円形の「ブロブ(blobs)」を形成します。重心の中心は、各状態の正確なノイズのないIQポイントを定義します。周囲のcloudは、様々なノイズソースから生成されたデータの分散を示しています。

|0>と|1>を区別するために、機械学習手法である線形判別分析を適用します。これは、qubit状態を分類するための一般的な手法です。

最初のステップは、重心データを取得することです。そのためには、2つのスケジュールを定義します。(システムが|0>状態であることを思い出してください。)

1. |0>の状態を直接測定します。(|0>の重心を取得)
2. $π$パルスを適用して測定します(|1>の重心を取得)

In [ ]:
# Create the two schedules

# Ground state schedule
zero_schedule = pulse.Schedule(name="zero schedule")
zero_schedule |= measure

# Excited state schedule
one_schedule = pulse.Schedule(name="one schedule")
one_schedule |= pi_pulse_01(drive_chan) 
one_schedule |= measure << one_schedule.duration

In [ ]:
# Assemble the schedules into a program
IQ_01_program = assemble([zero_schedule, one_schedule],
                          backend=backend,
                          meas_level=1,
                          meas_return='single',
                          shots=NUM_SHOTS,
                          schedule_los=[{drive_chan: cal_qubit_freq}] * 2)

In [ ]:
IQ_01_job = backend.run(IQ_01_program)

In [ ]:
print(IQ_01_job.job_id())
job_monitor(IQ_01_job)

In [ ]:
# Get job data (single); split for zero and one
IQ_01_data = get_job_data(IQ_01_job, average=False)
zero_data = IQ_01_data[0]
one_data = IQ_01_data[1]

In [ ]:
def IQ_01_plot(x_min, x_max, y_min, y_max):
    """Helper function for plotting IQ plane for |0>, |1>. Limits of plot given
    as arguments."""
    # zero data plotted in blue
    plt.scatter(np.real(zero_data), np.imag(zero_data), 
                    s=5, cmap='viridis', c='blue', alpha=0.5, label=r'$|0\rangle$')
    # one data plotted in red
    plt.scatter(np.real(one_data), np.imag(one_data), 
                    s=5, cmap='viridis', c='red', alpha=0.5, label=r'$|1\rangle$')

    # Plot a large dot for the average result of the zero and one states.
    mean_zero = np.mean(zero_data) # takes mean of both real and imaginary parts
    mean_one = np.mean(one_data)
    plt.scatter(np.real(mean_zero), np.imag(mean_zero), 
                s=200, cmap='viridis', c='black',alpha=1.0)
    plt.scatter(np.real(mean_one), np.imag(mean_one), 
                s=200, cmap='viridis', c='black',alpha=1.0)
    
    plt.xlim(x_min, x_max)
    plt.ylim(y_min,y_max)
    plt.legend()
    plt.ylabel('I [a.u.]', fontsize=15)
    plt.xlabel('Q [a.u.]', fontsize=15)
    plt.title("0-1 discrimination", fontsize=15)

以下に、IQプロットを表示します。青い重心は|0>状態を示し、赤い重心は|1>状態を示します。<br>
注：プロットが見えない場合は、ノートブックを再実行してください。

In [ ]:
x_min = -5
x_max = 15
y_min = -5
y_max = 10
IQ_01_plot(x_min, x_max, y_min, y_max)

さて、実際に弁別器を構築する時が来ました。上記のように、線形判別分析(LDA)と呼ばれる機械学習手法を使用します。LDAは、各カテゴリの平均間の距離を最大化し、各カテゴリ内の分散を最小化することにより、任意のデータセットを一連のカテゴリ(ここでは|0>、|1>)に分類します。詳細については、こちらを参照してください。<br>
https://scikit-learn.org/stable/modules/lda_qda.html#id4

LDAはセパラトリックスと呼ばれるラインを生成します。特定のデータポイントがセパラトリックスのどちら側にあるかに応じて、どのカテゴリに属しているかを判別できます。この例では、分離行列の片側が|0>状態に対応し、もう一方が|1>状態に対応しています。

データの前半を使用してモデルをトレーニングし、後半でテストします。LDAの実装にはscikit.learnを使用します。将来のリリースでは、この機能はQiskit-Ignisに直接リリースされて追加されます。(以下を参照)<br>
https://github.com/Qiskit/qiskit-ignis/tree/master/qiskit/ignis/measurement/discriminator

まず、結果データを識別に適した形式に再形成します。

In [ ]:
def reshape_complex_vec(vec):
    """Take in complex vector vec and return 2d array w/ real, imag entries. This is needed for the learning.
    Args:
        vec (list): complex vector of data
    Returns:
        list: vector w/ entries given by (real(vec], imag(vec))
    """
    length = len(vec)
    vec_reshaped = np.zeros((length, 2))
    for i in range(len(vec)):
        vec_reshaped[i]=[np.real(vec[i]), np.imag(vec[i])]
    return vec_reshaped

In [ ]:
# Create IQ vector (split real, imag parts)
zero_data_reshaped = reshape_complex_vec(zero_data)
one_data_reshaped = reshape_complex_vec(one_data)  

IQ_01_data = np.concatenate((zero_data_reshaped, one_data_reshaped))
print(IQ_01_data.shape) # verify IQ data shape

次に、トレーニングデータとテストデータを分割します。期待される結果(基底スケジュールは0の配列、励起スケジュールは1の配列)を含む状態ベクトルを使用してテストします。

In [ ]:
# construct vector w/ 0's and 1's (for testing)
state_01 = np.zeros(NUM_SHOTS) # shots gives number of experiments
state_01 = np.concatenate((state_01, np.ones(NUM_SHOTS)))
print(len(state_01))

# Shuffle and split data into training and test sets
IQ_01_train, IQ_01_test, state_01_train, state_01_test = train_test_split(IQ_01_data, state_01, test_size=0.5)

最後に、モデルを設定してトレーニングします。私たちのフィットの精度が印刷されています。

In [ ]:
# Set up the LDA
LDA_01 = LinearDiscriminantAnalysis()
LDA_01.fit(IQ_01_train, state_01_train)

In [ ]:
# test on some simple data 
print(LDA_01.predict([[0,0], [10, 0]]))

In [ ]:
# Compute accuracy
score_01 = LDA_01.score(IQ_01_test, state_01_test)
print(score_01)

最後のステップは、セパラトリックス(separatrix)をプロットすることです。

In [ ]:
# Plot separatrix on top of scatter
def separatrixPlot(lda, x_min, x_max, y_min, y_max, shots):
    nx, ny = shots, shots

    xx, yy = np.meshgrid(np.linspace(x_min, x_max, nx),
                         np.linspace(y_min, y_max, ny))
    Z = lda.predict_proba(np.c_[xx.ravel(), yy.ravel()])
    Z = Z[:, 1].reshape(xx.shape)

    plt.contour(xx, yy, Z, [0.5], linewidths=2., colors='black')

IQ_01_plot(x_min, x_max, y_min, y_max)
separatrixPlot(LDA_01, x_min, x_max, y_min, y_max, NUM_SHOTS)

セパラトリックスの片側がどのようにセントロイド(and a hence a state)に対応するかを確認します。IQ平面の点を指定すると、モデルはセパラトリックスのどちら側にあるかをチェックし、対応する状態を返します。

## 2. Discriminating the |0>, |1> and |2> states

0,1弁別器をキャリブレーションしたので、次に、より高いエネルギー状態を励起します。具体的には、|2>状態を励起し、それぞれのIQデータポイントから|0>、|1>および|2>状態を分類するための弁別子を構築することに焦点を当てています。さらに高い状態(|3>、|4>など)の手順も同様ですが、明示的にテストはしていません。

高いエネルギー状態を分類するための弁別子を構築するプロセスは次のとおりです。

1. $1→2$周波数を計算します。
2. Rabi実験を行って、$1→2$の$π$パルス振幅を取得します。これを行うには、最初に$0→1π$パルスを適用して、|0>から|1>状態にします。次に、上記で得られた$1→2$の周波数でドライブ振幅のスイープを実行します。
3. 3つのスケジュールを作成します。<br>
  1. 0スケジュール：基底状態を測定するだけです。
  2. 1スケジュール：$0→1π$パルスを適用して測定します。
  3. 2スケジュール：$0→1π$パルスを適用して測定し、次に$1→2π$パルスを適用して測定します。
4. 各スケジュールのデータをトレーニングセットとテストセットに分離し、差別化のためのLDAモデルを構築します。

### 2A. Computing the 1 -> 2 frequency

キャリブレーションの最初のステップは、$1→2$状態から移行するために必要な周波数を計算することです。これを行うには2つの方法があります。

1. 基底状態から周波数掃引を行い、非常に高い電力を適用します。適用した電力が十分に大きければ、2つのピークが観察されます。1つはセクション1で見つかった$0→1$周波数にあり、もう1つは$0→2$周波数にあります。$1→2$の周波数は、両者のさを取ることで得られます。残念ながら、ibmq_armonkの場合、この遷移を確認するには、最大駆動電力1.0では不十分です。代わりに、2番目の方法に移ります。<br>
2. $0→1π$パルスを適用して|1>状態を励起します。次に、|1>状態の励起に対して周波数スイープを実行します。単一ピークは、$1→2$周波数に対応する$0→1$周波数より低い周波数で観測されます。

#### 1 ->  2 サイドバンド法を使用した周波数スイープ

上記の2番目の方法に従います。$0→1π$パルスを駆動するには、キャリブレーションされた$0→1$周波数cal_qubit_freqによって与えられる局部発振器(LO)周波数が必要です。(セクション1のRabi$π$パルスの構成を参照)<br>
ただし、$1→2$周波数の範囲をスイープするには、LO周波数を変化させる必要があります。残念ながら、パルス使用では、スケジュールごとに1つのLO周波数が必要です。

これを解決するには、LO周波数をcal_qubit_freqに設定し、freq-cal_qubit_freqで$1→2$パルスに正弦関数を乗算します。ここで、freqは目的のスキャン周波数です。既知のように、正弦波サイドバンドを適用すると、プログラムの組み立て時に手動で設定することなくLO周波数を変更できます。

In [ ]:
def apply_sideband(pulse, freq):
    """Apply a sinusoidal sideband to this pulse at frequency freq.
    Args:
        pulse (SamplePulse): The pulse of interest.
        freq (float): LO frequency for which we want to apply the sweep.
    Return:
        SamplePulse: Pulse with a sideband applied (oscillates at difference between freq and cal_qubit_freq).
    """
    # time goes from 0 to dt*drive_samples, sine arg of form 2*pi*f*t
    t_samples = np.linspace(0, dt*drive_samples, drive_samples)
    sine_pulse = np.sin(2*np.pi*(freq-cal_qubit_freq)*t_samples) # no amp for the sine
    
    # create sample pulse w/ sideband applied
    # Note: need to make sq_pulse.samples real, multiply elementwise
    sideband_pulse = SamplePulse(np.multiply(np.real(pulse.samples), sine_pulse), name='sideband_pulse')
    
    return sideband_pulse    

プログラムを組み立てるためのロジックをメソッドにwrapして、プログラムを実行します。

In [ ]:
def create_excited_freq_sweep_program(freqs, drive_power):
    """Builds a program that does a freq sweep by exciting the |1> state. 
    This allows us to obtain the 1->2 frequency. We get from the |0> to |1>
    state via a pi pulse using the calibrated qubit frequency. To do the 
    frequency sweep from |1> to |2>, we use a sideband method by tacking
    a sine factor onto the sweep drive pulse.
    Args:
        freqs (np.ndarray(dtype=float)): Numpy array of frequencies to sweep.
        drive_power (float) : Value of drive amplitude.
    Raises:
        ValueError: Thrown if use more than 75 frequencies; currently, an error will be thrown on the backend 
                    if you try more than 75 frequencies.
    Returns:
        Qobj: Program for freq sweep experiment.
    """
    if len(freqs) > 75:
        raise ValueError("You can only run 75 schedules at a time.")
        
    print(f"The frequency sweep will go from {freqs[0] / GHz} GHz to {freqs[-1]/ GHz} GHz \
using {len(freqs)} frequencies. The drive power is {drive_power}.")

    base_12_pulse = pulse_lib.gaussian(duration=drive_samples,
                                        sigma=drive_sigma,
                                        amp=drive_power,
                                        name='base_12_pulse')
    schedules = []
    for jj, freq in enumerate(freqs):
        
        # add sideband to gaussian pulse
        freq_sweep_12_pulse = apply_sideband(base_12_pulse, freq)
        
        # add commands to schedule
        schedule = pulse.Schedule(name="Frequency = {}".format(freq))

        # Add 0->1 pulse, freq sweep pulse and measure
        schedule |= pi_pulse_01(drive_chan)
        schedule |= freq_sweep_12_pulse(drive_chan) << schedule.duration 
        schedule |= measure << schedule.duration # shift measurement to after drive pulses

        schedules.append(schedule)

    num_freqs = len(freqs)

    # draw a schedule
    display(schedules[-1].draw(channels_to_plot=[drive_chan, meas_chan], label=True, scaling=1.0))
    
    # assemble freq sweep program 
    # Note: LO is at cal_qubit_freq for each schedule; accounted for by sideband
    excited_freq_sweep_program = assemble(schedules,
                                          backend=backend, 
                                          meas_level=1,
                                          meas_return='avg',
                                          shots=NUM_SHOTS,
                                          schedule_los=[{drive_chan: cal_qubit_freq}]
                                                         * num_freqs)
    
    return excited_freq_sweep_program

In [ ]:
# sweep 400 MHz below 0->1 frequency to catch the 1->2 frequency
num_freqs = 75
excited_sweep_freqs = cal_qubit_freq + np.linspace(-400*MHz, 30*MHz, num_freqs)

In [ ]:
excited_freq_sweep_program = create_excited_freq_sweep_program(excited_sweep_freqs, drive_power=0.3)

# Plot an example schedule to make sure it's valid

In [ ]:
excited_freq_sweep_job = backend.run(excited_freq_sweep_program)

In [ ]:
print(excited_freq_sweep_job.job_id())
job_monitor(excited_freq_sweep_job)

In [ ]:
# Get job data (avg)
excited_freq_sweep_data = get_job_data(excited_freq_sweep_job, average=True)

In [ ]:
# Note: we are only plotting the real part of the signal
plt.scatter(excited_sweep_freqs/GHz, excited_freq_sweep_data, color='black')
plt.xlim([min(excited_sweep_freqs/GHz)+0.01, max(excited_sweep_freqs/GHz)]) # ignore min point (is off)
plt.xlabel("Frequency [GHz]", fontsize=15)
plt.ylabel("Measured Signal [a.u.]", fontsize=15)
plt.title("1->2 Frequency Sweep (first pass)", fontsize=15)
plt.show()

最小値は約4.64GHzです。いくつかの偽の最大値がありますが、$1→2$の周波数には大きすぎます。最小値は$1→2$の周波数に対応します。

相対最小関数を使用して、この点の値を正確に計算します。これにより、$1→2$周波数の推定値が得られます。

In [ ]:
# Prints out relative minima frequencies in output_data; height gives lower bound (abs val)
def rel_minima(freqs, output_data, height): 
    """
    Prints out relative minima frequencies in output_data (can see peaks); height gives upper bound (abs val).
    Be sure to set the height properly or the peak will be ignored!
    Args:
        freqs (list): frequency list
        output_data (list): list of resulting signals
        height (float): upper bound (abs val) on a peak
    Returns:
        list: List containing relative minima frequencies
    """
    peaks, _ = find_peaks(-1*output_data, height)
    print("Freq. dips: ", freqs[peaks])
    return freqs[peaks]

In [ ]:
minima = rel_minima(excited_sweep_freqs, np.real(excited_freq_sweep_data), 10)
approx_12_freq = minima[0]

ここで、上記で得られた推定値を使用して、洗練されたスイープを実行します。(つまり、範囲を大幅に狭めます)<br>
これにより、$1→2$周波数のより正確な値を取得できます。20MHzを各方向にスイープします。

In [ ]:
# smaller range refined sweep
num_freqs = 75
refined_excited_sweep_freqs = approx_12_freq + np.linspace(-20*MHz, 20*MHz, num_freqs)
refined_excited_freq_sweep_program = create_excited_freq_sweep_program(refined_excited_sweep_freqs, drive_power=0.3)

In [ ]:
refined_excited_freq_sweep_job = backend.run(refined_excited_freq_sweep_program)

In [ ]:
print(refined_excited_freq_sweep_job.job_id())
job_monitor(refined_excited_freq_sweep_job)

In [ ]:
# Get the refined data (average)
refined_excited_freq_sweep_data = get_job_data(refined_excited_freq_sweep_job, average=True)

標準のローレンツ曲線を使用して、洗練された信号をプロットして近似しましょう。

In [ ]:
# do fit in Hz
(refined_excited_sweep_fit_params, 
 refined_excited_sweep_y_fit) = fit_function(refined_excited_sweep_freqs,
                                     refined_excited_freq_sweep_data, 
                                     lambda x, A, q_freq, B, C: (A / np.pi) * (B / ((x - q_freq)**2 + B**2)) + C,
                                     [-12, 4.625*GHz, 0.05*GHz, 3*GHz] # initial parameters for curve_fit
                                     )

In [ ]:
# Note: we are only plotting the real part of the signal
plt.scatter(refined_excited_sweep_freqs/GHz, refined_excited_freq_sweep_data, color='black')
plt.plot(refined_excited_sweep_freqs/GHz, refined_excited_sweep_y_fit, color='red')
plt.xlim([min(refined_excited_sweep_freqs/GHz), max(refined_excited_sweep_freqs/GHz)])
plt.xlabel("Frequency [GHz]", fontsize=15)
plt.ylabel("Measured Signal [a.u.]", fontsize=15)
plt.title("1->2 Frequency Sweep (refined pass)", fontsize=15)
plt.show()

In [ ]:
_, qubit_12_freq, _, _ = refined_excited_sweep_fit_params
print(f"Our updated estimate for the 1->2 transition frequency is "
      f"{round(qubit_12_freq/GHz, 7)} GHz.")

### 2B. 1->2 Rabi Experiment

これで$1→2$周波数の推定が適切になったので、Rabi実験を実行して、$1→2$遷移の$π$パルス振幅を取得します。これを行うには、$0→1π$パルスを適用してから、(側波帯法を使用して)$1→2$周波数でドライブ振幅を掃引します。

In [ ]:
# experimental configuration
num_rabi_points = 75 # number of experiments (ie amplitudes to sweep out)

# Drive amplitude values to iterate over: 75 amplitudes evenly spaced from 0 to 1.0
drive_amp_min = 0
drive_amp_max = 1.0
drive_amps = np.linspace(drive_amp_min, drive_amp_max, num_rabi_points)

In [ ]:
# Create schedule
rabi_12_schedules = []

# loop over all drive amplitudes
for ii, drive_amp in enumerate(drive_amps):
    
    base_12_pulse = pulse_lib.gaussian(duration=drive_samples,
                                       sigma=drive_sigma,
                                       amp=drive_amp,
                                       name='base_12_pulse')
    # apply sideband at the 1->2 frequency
    rabi_12_pulse = apply_sideband(base_12_pulse, qubit_12_freq)
    
    # add commands to schedule
    schedule = pulse.Schedule(name='Rabi Experiment at drive amp = %s' % drive_amp)
    schedule |= pi_pulse_01(drive_chan) # 0->1
    schedule |= rabi_12_pulse(drive_chan) << schedule.duration # 1->2 Rabi pulse
    schedule |= measure << schedule.duration # shift measurement to after drive pulse
    
    rabi_12_schedules.append(schedule)

In [ ]:
# Assemble the schedules into a program
# Note: The LO frequency is at cal_qubit_freq to support the 0->1 pi pulse;
# it is modified for the 1->2 pulse using sidebanding
rabi_12_expt_program = assemble(rabi_12_schedules,
                                backend=backend,
                                meas_level=1,
                                meas_return='avg',
                                shots=NUM_SHOTS,
                                schedule_los=[{drive_chan: cal_qubit_freq}]
                                               * num_rabi_points)

In [ ]:
rabi_12_job = backend.run(rabi_12_expt_program)

In [ ]:
print(rabi_12_job.job_id())
job_monitor(rabi_12_job)

In [ ]:
# Get the job data (average)
rabi_12_data = get_job_data(rabi_12_job, average=True)

以前と同じようにデータをプロットして当てはめます。

In [ ]:
# Note: We only plot the real part of the signal.
rabi_12_data = np.real(baseline_remove(rabi_12_data))
(rabi_12_fit_params, 
 rabi_12_y_fit) = fit_function(drive_amps,
                            rabi_12_data, 
                            lambda x, A, B, drive_12_period, phi: (A*np.cos(2*np.pi*x/drive_12_period - phi) + B),
                            [3, 0.5, 0.9, 0])

plt.scatter(drive_amps, rabi_12_data, color='black')
plt.plot(drive_amps, rabi_12_y_fit, color='red')

drive_12_period = rabi_12_fit_params[2]
# account for phi in computing pi amp
pi_amp_12 = (drive_12_period/2/np.pi) *(np.pi+rabi_12_fit_params[3])

plt.axvline(pi_amp_12, color='red', linestyle='--')
plt.axvline(pi_amp_12+drive_12_period/2, color='red', linestyle='--')
plt.annotate("", xy=(pi_amp_12+drive_12_period/2, 0), xytext=(pi_amp_12,0), arrowprops=dict(arrowstyle="<->", color='red'))
plt.annotate("$\pi$", xy=(pi_amp_12-0.03, 0.1), color='red')

plt.xlabel("Drive amp [a.u.]", fontsize=15)
plt.ylabel("Measured signal [a.u.]", fontsize=15)
plt.title('Rabi Experiment (1->2)', fontsize=20)
plt.show()

In [ ]:
print(f"Pi Amplitude (1->2) = {pi_amp_12}")

この情報を使用して、$1→2π$パルスを定義できます。($1→2$周波数でサイドバンドを追加することを確認してください。)

In [ ]:
pi_pulse_12 = pulse_lib.gaussian(duration=drive_samples,
                                 amp=pi_amp_12, 
                                 sigma=drive_sigma,
                                 name='pi_pulse_12')
# make sure this pulse is sidebanded
pi_pulse_12 = apply_sideband(pi_pulse_12, qubit_12_freq)

### 2C. Build the 0,1,2 discriminator

最後に、|0>、|1>および|2>の状態に対する弁別子を作成します。手順はセクション1に似ていますが、|2>状態のスケジュールを追加します。

レビューとして、3つのスケジュールがあります。(ここでも、システムが|0>状態で開始することを思い出してください。)

1. |0>の状態を直接測定します。(|0>の重心を取得)
2. $0→1π$パルスを適用して測定します。(|1>セントロイドを取得)
3. $0→1π$パルスを適用し、次に$1→2π$パルスを適用して、測定します。(|2>セントロイドを取得)

In [ ]:
# Create the three schedules

# Ground state schedule
zero_schedule = pulse.Schedule(name="zero schedule")
zero_schedule |= measure

# Excited state schedule
one_schedule = pulse.Schedule(name="one schedule")
one_schedule |= pi_pulse_01(drive_chan) 
one_schedule |= measure << one_schedule.duration

# Excited state schedule
two_schedule = pulse.Schedule(name="two schedule")
two_schedule |= pi_pulse_01(drive_chan)
two_schedule |= pi_pulse_12(drive_chan) << two_schedule.duration
two_schedule |= measure << two_schedule.duration

In [ ]:
# Assemble the schedules into a program
IQ_012_program = assemble([zero_schedule, one_schedule, two_schedule],
                           backend=backend,
                           meas_level=1,
                           meas_return='single',
                           shots=NUM_SHOTS,
                           schedule_los=[{drive_chan: cal_qubit_freq}] * 3)

In [ ]:
IQ_012_job = backend.run(IQ_012_program)

In [ ]:
print(IQ_012_job.job_id())
job_monitor(IQ_012_job)

In [ ]:
# Get job data (single); split for zero, one and two
IQ_012_data = get_job_data(IQ_012_job, average=False)
zero_data = IQ_012_data[0]
one_data = IQ_012_data[1]
two_data = IQ_012_data[2]

In [ ]:
def IQ_012_plot(x_min, x_max, y_min, y_max):
    """Helper function for plotting IQ plane for 0, 1, 2. Limits of plot given
    as arguments."""
    # zero data plotted in blue
    plt.scatter(np.real(zero_data), np.imag(zero_data), 
                    s=5, cmap='viridis', c='blue', alpha=0.5, label=r'$|0\rangle$')
    # one data plotted in red
    plt.scatter(np.real(one_data), np.imag(one_data), 
                    s=5, cmap='viridis', c='red', alpha=0.5, label=r'$|1\rangle$')
    # two data plotted in green
    plt.scatter(np.real(two_data), np.imag(two_data), 
                    s=5, cmap='viridis', c='green', alpha=0.5, label=r'$|2\rangle$')

    # Plot a large dot for the average result of the 0, 1 and 2 states.
    mean_zero = np.mean(zero_data) # takes mean of both real and imaginary parts
    mean_one = np.mean(one_data)
    mean_two = np.mean(two_data)
    plt.scatter(np.real(mean_zero), np.imag(mean_zero), 
                s=200, cmap='viridis', c='black',alpha=1.0)
    plt.scatter(np.real(mean_one), np.imag(mean_one), 
                s=200, cmap='viridis', c='black',alpha=1.0)
    plt.scatter(np.real(mean_two), np.imag(mean_two), 
                s=200, cmap='viridis', c='black',alpha=1.0)
    
    plt.xlim(x_min, x_max)
    plt.ylim(y_min,y_max)
    plt.legend()
    plt.ylabel('I [a.u.]', fontsize=15)
    plt.xlabel('Q [a.u.]', fontsize=15)
    plt.title("0-1-2 discrimination", fontsize=15)

In [ ]:
x_min = -20
x_max = 10
y_min = -10
y_max = 5
IQ_012_plot(x_min, x_max, y_min, y_max)

|2>状態に対応する3番目の重心を観察します。<br>
注：プロットが見えない場合は、ノートブックを再実行してください。

このデータを使用して、弁別器を構築できます。ここでも、scikit.learnと線形判別分析(LDA)を使用します。

LDAのデータを形作ることから始めます。

In [ ]:
# Create IQ vector (split real, imag parts)
zero_data_reshaped = reshape_complex_vec(zero_data)
one_data_reshaped = reshape_complex_vec(one_data)  
two_data_reshaped = reshape_complex_vec(two_data)  

IQ_012_data = np.concatenate((zero_data_reshaped, one_data_reshaped, two_data_reshaped))
print(IQ_012_data.shape) # verify IQ data shape

次に、トレーニングデータとテストデータを分割します。(ここでも、半分と半分)<br>
テストデータは、0(ゼロスケジュールの場合)、1(1つのスケジュールの場合)及び2(2つのスケジュールの場合)の配列を含むベクトルです。

In [ ]:
# construct vector w/ 0's, 1's and 2's (for testing)
state_012 = np.zeros(NUM_SHOTS) # shots gives number of experiments
state_012 = np.concatenate((state_012, np.ones(NUM_SHOTS)))
state_012 = np.concatenate((state_012, 2*np.ones(NUM_SHOTS)))
print(len(state_012))

# Shuffle and split data into training and test sets
IQ_012_train, IQ_012_test, state_012_train, state_012_test = train_test_split(IQ_012_data, state_012, test_size=0.5)

最後に、モデルを設定してトレーニングします。私たちのフィットの精度が印刷されています。

In [ ]:
# Set up the LDA
LDA_012 = LinearDiscriminantAnalysis()
LDA_012.fit(IQ_012_train, state_012_train)

In [ ]:
# test on some simple data 
print(LDA_012.predict([[0, 0], [-10, 0], [-15, -5]]))

In [ ]:
# Compute accuracy
score_012 = LDA_012.score(IQ_012_test, state_012_test)
print(score_012)

最後のステップは、セパラトリックスをプロットすることです。

In [ ]:
IQ_012_plot(x_min, x_max, y_min, y_max)
separatrixPlot(LDA_012, x_min, x_max, y_min, y_max, NUM_SHOTS)

これで3つの図心ができたので、セパラトリックスは線ではなく、2つの線の組み合わせを含む曲線になります。|0>、|1>、|2>の状態を区別するために、モデルでは、IQポイントが分離行列に対してどこにあるかをチェックし、それに応じてポイントを分類します。